In [1]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict, namedtuple, OrderedDict
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution
import os
from io import BytesIO
from itertools import chain
import random

In [2]:
def read_data(filename):
    """Read tagged sentence data"""
    with open(filename, 'r', encoding='utf-8', errors='ignore') as f:
        sentence_lines = [l.split("\n") for l in f.read().split("\n\n")]
    return OrderedDict(((s[0], Sentence(*zip(*[l.strip().split("\t")
                        for l in s[1:]]))) for s in sentence_lines if s[0]))

def read_tags(filename):
    """Read a list of word tag classes"""
    with open(filename, 'r') as f:
        tags = f.read().split("\n")
    return frozenset(tags)

Sentence = namedtuple("Sentence", "words tags")

In [32]:
tag = read_tags("english9.txt")
sentence1 = read_data("hindi9.txt")
sentence1

OrderedDict([('b100-5507',
              Sentence(words=('इसके   ', 'अतिरिक्त', 'गुग्गुल', 'कुंड', 'भीम', 'गुफा ', 'तथा', 'भीमशिला', 'भी', 'दर्शनीय', 'है', 'इसके   ', 'अतिरिक्त', 'गुग्गुल', 'कुंड', 'भीम', 'गुफा ', 'तथा', 'भीमशिला', 'भी', 'दर्शनीय', 'है'), tags=('Apart', 'from this', 'Guggul', 'Kund', 'Bhima', 'Cave', 'and', 'Bhimshila', 'also worth', 'visiting', 'is', 'Apart', 'from this', 'Guggul', 'Kund', 'Bhima', 'Cave', 'and', 'Bhimshila', 'also worth', 'visiting', 'is'))),
             ('b100-935',
              Sentence(words=('इसके   ', 'अतिरिक्त', 'गुग्गुल', 'कुंड', 'भीम', 'गुफा ', 'तथा', 'भीमशिला', 'भी', 'दर्शनीय', 'है', 'आधा', 'किमी ', 'की ', 'दूरी ', 'पर ', 'भैरवनाथ ', 'मंदिर ', 'जहाँ ', 'केवल ', 'केदारनाथ ', 'के ', 'पट ', 'खुलने ', 'और ', 'बंद ', 'होने ', 'के ', 'दिन ', 'ही ', 'पूजन ', 'किया ', 'जाता ', 'है'), tags=('Apart', 'from this', 'Guggul', 'Kund', 'Bhima', 'Cave', 'and', 'Bhimshila', 'also worth', 'visiting', 'is', 'half', 'km', 'a', 'distance', 'at', 'Bhairavnath',

In [33]:
class Dataset(namedtuple("_Dataset", "sentences keys vocab X tagset Y training_set testing_set N stream")):
    def __new__(cls, tagfile, datafile, train_test_split=0.8, seed=112890):
        tagset = read_tags(tagfile)
        sentences = read_data(datafile)
        keys = tuple(sentences.keys())
        wordset = frozenset(chain(*[s.words for s in sentences.values()]))
        word_sequences = tuple([sentences[k].words for k in keys])
        tag_sequences = tuple([sentences[k].tags for k in keys])
        N = sum(1 for _ in chain(*(s.words for s in sentences.values())))
        
        # split data into train/test sets
        _keys = list(keys)
        if seed is not None: random.seed(seed)
        random.shuffle(_keys)
        split = int(train_test_split * len(_keys))
        training_data = Subset(sentences, _keys[:split])
        testing_data = Subset(sentences, _keys[split:])
        stream = tuple(zip(chain(*word_sequences), chain(*tag_sequences)))
        return super().__new__(cls, dict(sentences), keys, wordset, word_sequences, tagset,
                               tag_sequences, training_data, testing_data, N, stream.__iter__)

    def __len__(self):
        return len(self.sentences)

    def __iter__(self):
        return iter(self.sentences.items())
    
    
class Subset(namedtuple("BaseSet", "sentences keys vocab X tagset Y N stream")):
    def __new__(cls, sentences, keys):
        word_sequences = tuple([sentences[k].words for k in keys])
        tag_sequences = tuple([sentences[k].tags for k in keys])
        wordset = frozenset(chain(*word_sequences))
        tagset = frozenset(chain(*tag_sequences))
        N = sum(1 for _ in chain(*(sentences[k].words for k in keys)))
        stream = tuple(zip(chain(*word_sequences), chain(*tag_sequences)))
        return super().__new__(cls, {k: sentences[k] for k in keys}, keys, wordset, word_sequences,
                               tagset, tag_sequences, N, stream.__iter__)

    def __len__(self):
        return len(self.sentences)

    def __iter__(self):
        return iter(self.sentences.items())

In [36]:
data = Dataset("english9.txt", "hindi9.txt", train_test_split=0.8)

print("There are {} sentences in the corpus.".format(len(data)))
print("There are {} sentences in the training set.".format(len(data.training_set)))
print("There are {} sentences in the testing set.".format(len(data.testing_set)))

assert len(data) == len(data.training_set) + len(data.testing_set), \
       "The number of sentences in the training set + testing set should sum to the number of sentences in the corpus"

There are 18 sentences in the corpus.
There are 14 sentences in the training set.
There are 4 sentences in the testing set.


In [37]:
for i in range(2):    
    print("Sentence {}:".format(i + 1), data.X[i])
    print()
    print("Labels {}:".format(i + 1), data.Y[i])
    print()

Sentence 1: ('इसके   ', 'अतिरिक्त', 'गुग्गुल', 'कुंड', 'भीम', 'गुफा ', 'तथा', 'भीमशिला', 'भी', 'दर्शनीय', 'है', 'इसके   ', 'अतिरिक्त', 'गुग्गुल', 'कुंड', 'भीम', 'गुफा ', 'तथा', 'भीमशिला', 'भी', 'दर्शनीय', 'है')

Labels 1: ('Apart', 'from this', 'Guggul', 'Kund', 'Bhima', 'Cave', 'and', 'Bhimshila', 'also worth', 'visiting', 'is', 'Apart', 'from this', 'Guggul', 'Kund', 'Bhima', 'Cave', 'and', 'Bhimshila', 'also worth', 'visiting', 'is')

Sentence 2: ('इसके   ', 'अतिरिक्त', 'गुग्गुल', 'कुंड', 'भीम', 'गुफा ', 'तथा', 'भीमशिला', 'भी', 'दर्शनीय', 'है', 'आधा', 'किमी ', 'की ', 'दूरी ', 'पर ', 'भैरवनाथ ', 'मंदिर ', 'जहाँ ', 'केवल ', 'केदारनाथ ', 'के ', 'पट ', 'खुलने ', 'और ', 'बंद ', 'होने ', 'के ', 'दिन ', 'ही ', 'पूजन ', 'किया ', 'जाता ', 'है')

Labels 2: ('Apart', 'from this', 'Guggul', 'Kund', 'Bhima', 'Cave', 'and', 'Bhimshila', 'also worth', 'visiting', 'is', 'half', 'km', 'a', 'distance', 'at', 'Bhairavnath', 'temple', 'where', 'only', 'Kedarnath', 'is', 'door', 'opening', 'and', 'closi

In [39]:
key = 'b100-5507'
print("Sentence: {}".format(key))
print("words:\n\t{!s}".format(data.sentences[key].words))
print("tags:\n\t{!s}".format(data.sentences[key].tags))

Sentence: b100-5507
words:
	('इसके   ', 'अतिरिक्त', 'गुग्गुल', 'कुंड', 'भीम', 'गुफा ', 'तथा', 'भीमशिला', 'भी', 'दर्शनीय', 'है', 'इसके   ', 'अतिरिक्त', 'गुग्गुल', 'कुंड', 'भीम', 'गुफा ', 'तथा', 'भीमशिला', 'भी', 'दर्शनीय', 'है')
tags:
	('Apart', 'from this', 'Guggul', 'Kund', 'Bhima', 'Cave', 'and', 'Bhimshila', 'also worth', 'visiting', 'is', 'Apart', 'from this', 'Guggul', 'Kund', 'Bhima', 'Cave', 'and', 'Bhimshila', 'also worth', 'visiting', 'is')


In [40]:
words = [word for i, (word, tag) in enumerate(data.training_set.stream())]
tags = [tag for i, (word, tag) in enumerate(data.training_set.stream())]
words[0:4], tags[0:4]

(['सभी ', 'पर्व ', 'ओर ', 'त्योहार '], ['All', 'festivals', 'and', 'festival'])

In [41]:
def pair_counts(tags, words):
    d = defaultdict(lambda: defaultdict(int))
    for tag, word in zip(tags, words):
        d[tag][word] += 1
    return d
        
word_counts = pair_counts(words, tags)

In [42]:
mfc_table = dict((word, max(tags.keys(), key=lambda key: tags[key])) for word, tags in word_counts.items())

In [43]:
i = 0
for key, value in mfc_table.items():
    print(key, value)
    i += 1
    if i > 3: break

सभी  All
पर्व  festivals
ओर  and
त्योहार  festival


In [44]:
FakeState = namedtuple('FakeState', 'name')

class MFCTagger:
    missing = FakeState(name = '<MISSING>')
    
    def __init__(self, table):
        self.table = defaultdict(lambda: MFCTagger.missing)
        self.table.update({word: FakeState(name=tag) for word, tag in table.items()})
        
    def viterbi(self, seq):
        """This method simplifies predictions by matching the Pomegranate viterbi() interface"""
        return 0., list(enumerate(["<start>"] + [self.table[w] for w in seq] + ["<end>"]))

In [45]:
# Using Most frequent POS tag
mfc_model = MFCTagger(mfc_table)

In [46]:
def replace_unknown(sequence):
    return [w if w in data.training_set.vocab else 'nan' for w in sequence]

def simplify_decoding(X, model):    
    _, state_path = model.viterbi(replace_unknown(X))
    return [state[1].name for state in state_path[1:-1]]

In [47]:
for key in data.testing_set.keys[:2]:
    print("Sentence Key: {}\n".format(key))
    print("Sentence: {}\n".format(data.sentences[key].words))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, mfc_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-27528

Sentence: ('इराक  ', 'के', 'विदेश', 'मंत्री', 'अमरीका', 'के', 'उस', 'प्रस्ताव', 'मजाक', 'उड़ाया', 'जिसमें', 'अमरीका', 'संयुक्त', 'राष्ट्र', 'के', 'प्रतिबंधों', 'इराकी', 'नागरिकों', 'के', 'लिए', 'कम', 'हानिकारक', 'बनाने', 'के', 'लिए', 'कहा', 'आतंकवाद', 'एक', 'ऐसी', 'समस्या', 'जिसने', 'न', 'केवल', 'भारत', 'अपितु', 'पूरे', 'विश्व', 'को', 'अपने', 'लपेटे', 'ले', 'रखा', 'है')

Predicted labels:
-----------------
['Iraq', 'for', 'foreign', 'minister', 'america', 'for', 'that', 'proposal', 'mock', 'made', 'in', 'america', 'united', 'nations', 'for', 'sanctions', 'Iraq', 'citizens', 'for', 'them', 'less', 'harmful', 'make', 'for', 'them', 'said', 'Terrorism', 'one', 'that', 'problem', 'which', 'not', 'only', 'India', 'but', 'whole', 'world', 'for', 'our', 'engulfed', 'take', 'kept', 'is']

Actual labels:
--------------
('Iraq', 'for', 'foreign', 'minister', 'america', 'for', 'that', 'proposal', 'mock', 'made', 'in', 'america', 'united', 'nations', 'for', 'sanctions', 'I

In [48]:
def accuracy(X, Y, model):
    
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        
        # The model.viterbi call in simplify_decoding will return None if the HMM
        # raises an error (for example, if a test sentence contains a word that
        # is out of vocabulary for the training set). Any exception counts the
        # full sentence as an error (which makes this a conservative estimate).
        try:
            most_likely_tags = simplify_decoding(observations, model)
            correct += sum(p == t for p, t in zip(most_likely_tags, actual_tags))
        except:
            pass
        total_predictions += len(observations)
    return correct / total_predictions

In [49]:
mfc_training_acc = accuracy(data.training_set.X, data.training_set.Y, mfc_model)
print("training accuracy mfc_model: {:.2f}%".format(100 * mfc_training_acc))

mfc_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, mfc_model)
print("testing accuracy mfc_model: {:.2f}%".format(100 * mfc_testing_acc))

training accuracy mfc_model: 92.65%
testing accuracy mfc_model: 93.94%


In [50]:
def unigram_counts(sequences):
    return Counter(sequences)

tags = [tag for i, (word, tag) in enumerate(data.training_set.stream())]
tag_unigrams = unigram_counts(tags)
tag_unigrams

Counter({'All': 6,
         'festivals': 18,
         'and': 31,
         'festival': 32,
         'mutual': 12,
         'love': 6,
         'brotherhood': 6,
         'reconciliation': 6,
         'meet': 6,
         'message': 6,
         'gives': 12,
         'does': 12,
         'other': 6,
         'words': 6,
         'in': 14,
         'all': 8,
         'get': 12,
         'we': 6,
         'virtue': 6,
         'sympathy': 6,
         'cooperation': 6,
         'humanity': 6,
         'it': 43,
         'sense': 6,
         'worth': 6,
         'noting': 12,
         'is': 75,
         'That': 6,
         'any': 13,
         'or': 32,
         'indigenous': 6,
         'be': 30,
         'foreign': 11,
         'poor': 6,
         'rich': 6,
         'class': 6,
         'whether': 6,
         'above': 6,
         'characteristic': 6,
         'superiority': 6,
         'must': 6,
         'have': 6,
         'Christmas': 7,
         'December': 6,
         "'s": 2,
         

In [51]:
def bigram_counts(sequences):
    return Counter(sequences)

tags = [tag for i, (word, tag) in enumerate(data.stream())]
o = [(tags[i],tags[i+1]) for i in range(0,len(tags)-2,2)]
tag_bigrams = bigram_counts(o)
tag_bigrams 

Counter({('Apart', 'from this'): 4,
         ('Guggul', 'Kund'): 4,
         ('Bhima', 'Cave'): 4,
         ('and', 'Bhimshila'): 4,
         ('also worth', 'visiting'): 4,
         ('is', 'Apart'): 3,
         ('from this', 'Guggul'): 3,
         ('Kund', 'Bhima'): 3,
         ('Cave', 'and'): 3,
         ('Bhimshila', 'also worth'): 3,
         ('visiting', 'is'): 3,
         ('is', 'half'): 2,
         ('km', 'a'): 4,
         ('distance', 'at'): 4,
         ('Bhairavnath', 'temple'): 4,
         ('where', 'only'): 4,
         ('Kedarnath', 'is'): 4,
         ('door', 'opening'): 4,
         ('and', 'closing'): 4,
         ('done', 'is'): 7,
         ('day', 'only'): 4,
         ('worship', 'done'): 4,
         ('is', 'done'): 4,
         ('half', 'km'): 3,
         ('a', 'distance'): 3,
         ('at', 'Bhairavnath'): 3,
         ('temple', 'where'): 3,
         ('only', 'Kedarnath'): 3,
         ('is', 'door'): 3,
         ('opening', 'and'): 3,
         ('closing', 'done'): 3,
  

In [52]:
def starting_counts(sequences):
    return Counter(sequences)

tags = [tag for i, (word, tag) in enumerate(data.stream())]
starts_tag = [i[0] for i in data.Y]
tag_starts = starting_counts(starts_tag)
tag_starts

Counter({'Apart': 2,
         'half': 1,
         'All': 1,
         'Iraq': 5,
         'Christmas': 6,
         'Bhairav': 1,
         'one': 1,
         'for': 1})

In [53]:
def ending_counts(sequences):    
    return Counter(sequences)

end_tag = [i[len(i)-1] for i in data.Y]
tag_ends = ending_counts(end_tag)
tag_ends

Counter({'is': 3, 'done': 2, 'it': 6, 'has': 5, 'any': 1, 'said': 1})

In [54]:
end_tag = [i[len(i)-2] for i in data.Y]
tag_ends = ending_counts(end_tag)
tag_ends

Counter({'visiting': 1,
         'is': 2,
         'important': 1,
         'have': 6,
         'kept': 5,
         'without': 1,
         'them': 1,
         'festival': 1})

In [55]:
hmm_model = HiddenMarkovModel(name="base-hmm-tagger")

tags = [tag for i, (word, tag) in enumerate(data.stream())]
words = [word for i, (word, tag) in enumerate(data.stream())]

tags_count = unigram_counts(tags)
tag_words_count = pair_counts(tags, words)

starting_tag_list = [i[0] for i in data.Y]
#ending_tag_list = [i[-1] if len(i)==1 else i[-2] for i in data.Y]
#ending_tag_list = [i[-1] for i in data.Y]
ending_tag_list = [i[len(i)-1] for i in data.Y]

starting_tag_count = starting_counts(starting_tag_list) #the number of times a tag occured at the start
ending_tag_count = ending_counts(ending_tag_list)       #the number of times a tag occured at the end

tag_words_count

defaultdict(<function __main__.pair_counts.<locals>.<lambda>()>,
            {'Apart': defaultdict(int, {'इसके   ': 7}),
             'from this': defaultdict(int, {'अतिरिक्त': 7}),
             'Guggul': defaultdict(int, {'गुग्गुल': 7}),
             'Kund': defaultdict(int, {'कुंड': 7}),
             'Bhima': defaultdict(int, {'भीम': 7}),
             'Cave': defaultdict(int, {'गुफा ': 7}),
             'and': defaultdict(int,
                         {'तथा': 7, 'और ': 14, 'ओर ': 14, 'ओर': 1}),
             'Bhimshila': defaultdict(int, {'भीमशिला': 7}),
             'also worth': defaultdict(int, {'भी': 7}),
             'visiting': defaultdict(int, {'दर्शनीय': 7}),
             'is': defaultdict(int,
                         {'है': 23,
                          'के ': 21,
                          'जाता ': 7,
                          'का ': 14,
                          'की ': 7,
                          'हैं': 7,
                          'के': 1,
                          'जाता'

In [56]:
to_pass_states = []
for tag, words_dict in tag_words_count.items():
    total = float(sum(words_dict.values()))
    distribution = {word: count/total for word, count in words_dict.items()}
    tag_emissions = DiscreteDistribution(distribution)
    tag_state = State(tag_emissions, name=tag)
    to_pass_states.append(tag_state)

In [57]:
distribution

{'भेदभाव': 1.0}

In [58]:
to_pass_states

[{
     "class" : "State",
     "distribution" : {
         "class" : "Distribution",
         "dtype" : "str",
         "name" : "DiscreteDistribution",
         "parameters" : [
             {
                 "\u0907\u0938\u0915\u0947   " : 1.0
             }
         ],
         "frozen" : false
     },
     "name" : "Apart",
     "weight" : 1.0
 }, {
     "class" : "State",
     "distribution" : {
         "class" : "Distribution",
         "dtype" : "str",
         "name" : "DiscreteDistribution",
         "parameters" : [
             {
                 "\u0905\u0924\u093f\u0930\u093f\u0915\u094d\u0924" : 1.0
             }
         ],
         "frozen" : false
     },
     "name" : "from this",
     "weight" : 1.0
 }, {
     "class" : "State",
     "distribution" : {
         "class" : "Distribution",
         "dtype" : "str",
         "name" : "DiscreteDistribution",
         "parameters" : [
             {
                 "\u0917\u0941\u0917\u094d\u0917\u0941\u0932" : 1.0
  

In [59]:
start_prob={}

for tag in tags:
    start_prob[tag] = starting_tag_count[tag] / tags_count[tag]

for tag_state in to_pass_states :
    hmm_model.add_transition(hmm_model.start, tag_state, start_prob[tag_state.name]) 

In [60]:
end_prob={}

for tag in tags:
    end_prob[tag] = ending_tag_count[tag]/tags_count[tag]
    
for tag_state in to_pass_states :
    hmm_model.add_transition(tag_state, hmm_model.end, end_prob[tag_state.name])

In [61]:
transition_prob_pair={}

for key in tag_bigrams.keys():
    transition_prob_pair[key] = tag_bigrams.get(key)/tags_count[key[0]]
    
for tag_state in to_pass_states:
    for next_tag_state in to_pass_states:
        if((tag_state.name,next_tag_state.name) in transition_prob_pair):
            hmm_model.add_transition(tag_state, next_tag_state, transition_prob_pair[(tag_state.name, next_tag_state.name)])

In [62]:
hmm_model.bake()

In [63]:
hmm_training_acc = accuracy(data.training_set.X, data.training_set.Y, hmm_model)
print("training accuracy basic hmm model: {:.2f}%".format(100 * hmm_training_acc))

hmm_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, hmm_model)
print("testing accuracy basic hmm model: {:.2f}%".format(100 * hmm_testing_acc))

training accuracy basic hmm model: 95.37%
testing accuracy basic hmm model: 100.00%
